In [1]:
import json
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

# Load the scaler parameters from JSON
with open('/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/Materials_NN/Paper_Materials_2025/training/batch_3/scaler_y_0_05eV.json', 'r') as f:
    scaler_params = json.load(f)

# Create a new MinMaxScaler instance and set its parameters
scaler_y = MinMaxScaler(feature_range=tuple(scaler_params['feature_range']))
scaler_y.min_ = np.array(scaler_params['min_'])
scaler_y.scale_ = np.array(scaler_params['scale_'])
scaler_y.data_min_ = np.array(scaler_params['data_min_'])
scaler_y.data_max_ = np.array(scaler_params['data_max_'])
scaler_y.data_range_ = np.array(scaler_params['data_range_'])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

class DeepSet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSet, self).__init__()
        self.rho = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        output = self.rho(x)
        return output

class InverseScaler(nn.Module):
    def __init__(self, feature_range, data_min, data_range):
        super(InverseScaler, self).__init__()
        feature_min, feature_max = feature_range
        self.a = data_range / (feature_max - feature_min)
        self.b = (-feature_min * self.a) + data_min
        # Convert to tensors and register as buffers
        self.register_buffer('a_tensor', torch.tensor(self.a, dtype=torch.float32))
        self.register_buffer('b_tensor', torch.tensor(self.b, dtype=torch.float32))
    
    def forward(self, x):
        return x * self.a_tensor + self.b_tensor

class DeepSetWithInverseScaling(nn.Module):
    def __init__(self, original_model, scaler_params):
        super(DeepSetWithInverseScaling, self).__init__()
        self.original_model = original_model
        feature_range = scaler_params['feature_range']
        data_min = scaler_params['data_min_'][0]
        data_range = scaler_params['data_range_'][0]
        self.inverse_scaler = InverseScaler(feature_range, data_min, data_range)
    
    def forward(self, x):
        scaled_output = self.original_model(x)
        inverse_scaled_output = self.inverse_scaler(scaled_output)
        return inverse_scaled_output

# Load OH_elements and p_dict_oh as in your original code
with open('/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/Materials_NN/Paper_Materials_2025/training/OH_elements.json', 'r') as file:
    OH_elements = json.load(file)

total_N_elements = len(OH_elements)

with open('/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/Materials_NN/Paper_Materials_2025/training/p_dict_oh.json', 'r') as file:
    p_dict_oh = json.load(file)

# Parameters
input_dim = len(p_dict_oh['H'])  # Dimension of each feature vector
hidden_dim = 512  # Hidden layer size
output_dim = 1  # Single value output (formation energy)

# Initialize the original model and load state
model = DeepSet(input_dim, hidden_dim, output_dim).to(device)
model_path = '/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/Materials_NN/Paper_Materials_2025/training/batch_3/model_tanh_wd_1e-8_3layer_0_05eV.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Prepare scaler parameters for inverse scaling
scaler_params_inverse = {
    'feature_range': scaler_y.feature_range,
    'data_min_': scaler_y.data_min_,
    'data_range_': scaler_y.data_range_
}

# Create the combined model
combined_model = DeepSetWithInverseScaling(model, scaler_params_inverse).to(device)
combined_model.eval()



Using device: cpu


DeepSetWithInverseScaling(
  (original_model): DeepSet(
    (rho): Sequential(
      (0): Linear(in_features=118, out_features=512, bias=True)
      (1): Tanh()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): Tanh()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): Tanh()
      (6): Linear(in_features=512, out_features=1, bias=True)
    )
  )
  (inverse_scaler): InverseScaler()
)

In [4]:
# Biases: Materials Project minus Experimental formation energy (eV/fu)
bias = {}
bias['Mn'] = -3.958 + 3.528
bias['Zn'] = -3.286 + 2.954
bias['Ti'] = -9.948 + 9.213


species_data = {}

# Database of experimental free energies plus Materials Project bias (eV/fu)

species_data = [
    {
        'name': 'Mn$^{2+}$',
        'formula_str': 'Mn',
        'phase': 'ion',
        'charge': +2,
        'Ef': -2.387 + bias['Mn']
    },
    {
        'name': 'MnO$_4$$^{2-}$',
        'formula_str': 'MnO4',
        'phase': 'ion',
        'charge': -2,
        'Ef': -5.222 + bias['Mn']
    },
    {
        'name': 'MnHO$_2$$^{-}$',
        'formula_str': 'MnHO2',
        'phase': 'ion',
        'charge': -1,
        'Ef': -5.243 + bias['Mn']
    },
    {
        'name': 'Mn$^{3+}$',
        'formula_str': 'Mn',
        'phase': 'ion',
        'charge': +3,
        'Ef': -0.850 + bias['Mn']
    },  
    {
        'name': 'MnO$_4$$^{-}$',
        'formula_str': 'MnO4',
        'phase': 'ion',
        'charge': -1,
        'Ef': -4.658 + bias['Mn']
    },  
    {
        'name': 'MnO',
        'formula_str': 'MnO',
        'phase': 'solid',
        'charge': 0,
        'Ef': -3.958
    },
    {
        'name': 'Mn',
        'formula_str': 'Mn',
        'phase': 'solid',
        'charge': 0,
        'Ef': 0.0
    },


    {
        'name': 'Zn$^{2+}$',
        'formula_str': 'Zn',
        'phase': 'ion',
        'charge': +2,
        'Ef': -1.525 + bias['Zn']
    },    
    {
        'name': 'ZnO$_2$$^{aq}$',
        'formula_str': 'ZnO2',
        'phase': 'ion',
        'charge': 0,
        'Ef': -2.921 + bias['Zn']
    }, 
    {
        'name': 'ZnOH$^{+}$',
        'formula_str': 'ZnOH',
        'phase': 'ion',
        'charge': +1,
        'Ef': -3.518 + bias['Zn']
    },     
    {
        'name': 'ZnO$_2$$^{2-}$',
        'formula_str': 'ZnO2',
        'phase': 'ion',
        'charge': -2,
        'Ef': -4.042 + bias['Zn']
    }, 
    {
        'name': 'ZnHO$_2$$^{-}$',
        'formula_str': 'ZnHO2',
        'phase': 'ion',
        'charge': -1,
        'Ef': -4.810 + bias['Zn']
    }, 
    {
        'name': 'ZnO',
        'formula_str': 'ZnO',
        'phase': 'solid',
        'charge': 0,
        'Ef': -3.286
    },
    {
        'name': 'Zn',
        'formula_str': 'Zn',
        'phase': 'solid',
        'charge': 0,
        'Ef': 0.0
    }, 


    {
        'name': 'Ti$^{2+}$',
        'formula_str': 'Ti',
        'phase': 'ion',
        'charge': +2,
        'Ef': -3.257 + bias['Ti']
    },
    {
        'name': 'Ti$^{3+}$',
        'formula_str': 'Ti',
        'phase': 'ion',
        'charge': +3,
        'Ef': -3.626 + bias['Ti']
    },
    {
        'name': 'TiO$^{2+}$',
        'formula_str': 'TiO',
        'phase': 'ion',
        'charge': +2,
        'Ef': -4.843 + bias['Ti']
    },
    {
        'name': 'TiHO$_3$$^{-}$',
        'formula_str': 'TiHO3',
        'phase': 'ion',
        'charge': -1,
        'Ef': -9.908 + bias['Ti']
    },
    {
        'name': 'TiO$_2$$^{2+}$',
        'formula_str': 'TiO2',
        'phase': 'ion',
        'charge': +2,
        'Ef': -4.845 + bias['Ti']
    },
    {
        'name': 'TiO$_2$',
        'formula_str': 'TiO2',
        'phase': 'solid',
        'charge': 0,
        'Ef': -9.948
    },
    {
        'name': 'Ti',
        'formula_str': 'Ti',
        'phase': 'solid',
        'charge': 0,
        'Ef': 0.0
    }
]


import re

# Function to parse chemical formulas
def parse_formula(formula):
    # Updated regex pattern to capture decimal numbers
    pattern = r'([A-Z][a-z]*)(\d*\.?\d*)'
    tokens = re.findall(pattern, formula)
    formula_dict = {}
    for (element, count) in tokens:
        if count == '':
            count = 1.0  # Default to 1.0 for elements without a count
        else:
            count = float(count)  # Convert to float to handle decimal numbers
        formula_dict[element] = formula_dict.get(element, 0) + count
    return formula_dict

# Parse the formulas
for data in species_data:
    if 'formula_str' in data:
        data['formula'] = parse_formula(data['formula_str'])
    else:
        data['formula'] = parse_formula(data['name'])


# Function to calculate Gibbs free energy for each species at each point
def get_dG(species_A, species_B, conc_A, conc_B, Gf_AB_alloy, pH, V):
    """
    Calculate gibbs energy for given binary alloy where one element decomposes into an ion.

    """
    # Get number of O, H, and charge in the two compounds
    yA = species_A['formula'].get('O', 0)
    zA = species_A['formula'].get('H', 0)
    cA = species_A['charge']

    yB = species_B['formula'].get('O', 0)
    zB = species_B['formula'].get('H', 0)
    cB = species_B['charge']


    # Get number of H2O, H^+, e^-
    nH2O = conc_A * yA + conc_B * yB
    nH = 2 * nH2O - (conc_A * zA + conc_B * zB)
    nEl = nH + conc_A * cA + conc_B * cB

    # Get free energies
    GfA = species_A['Ef'] # in eV/fu
    GfB = species_B['Ef'] # in eV/fu
    GfH2O = -2.46      # in eV/fu
    GfAB = Gf_AB_alloy # in eV/fu

    # Ion indicators
    if cA == 0:
        kA = 0
    else:
        kA = 1
    
    if cB == 0:
        kB = 0
    else:
        kB = 1

    # Claculate modifed free energy
    dG = conc_A * GfA + conc_B * GfB - GfAB - nH2O * GfH2O + 0.0591 * (conc_A * kA * np.log10(10**(-6)) + conc_B * kB * np.log10(10**(-6))) - 0.0591 * nH * pH - nEl * V

    return dG

def get_Gf_oxide(el_A, el_B, conc_A, conc_B, gamma, model, p_dict_oh_t):

    x_A = p_dict_oh_t[el_A]
    x_B = p_dict_oh_t[el_B]
    x_O = p_dict_oh_t['O']

    x = (1 - gamma) * conc_A * x_A + (1 - gamma) * conc_B * x_B + gamma * x_O

    E = model(x)

    # grad_x = torch.autograd.grad(
    #     outputs=E,
    #     inputs=x,
    #     create_graph=True,
    #     retain_graph=True
    # )[0]

    return E.item()

def get_Gf_alloy(el_A, el_B, conc_A, conc_B, model, p_dict_oh_t):

    x_A = p_dict_oh_t[el_A]
    x_B = p_dict_oh_t[el_B]

    x = conc_A * x_A + conc_B * x_B

    E = model(x)

    # grad_x = torch.autograd.grad(
    #     outputs=E,
    #     inputs=x,
    #     create_graph=True,
    #     retain_graph=True
    # )[0]

    return E.item()

def oxide_opt_obj_fun(x, el_A, el_B, conc_A, conc_B, model, p_dict_oh_t, pH, V):

    energy_O = get_Gf_oxide(el_A, el_B, conc_A, conc_B, x, model, p_dict_oh_t)
    energy = get_Gf_alloy(el_A, el_B, conc_A, conc_B, model, p_dict_oh_t)

    Gf_H2O = -2.46 

    fun = 1 / (1 - x) * energy_O - energy - x / (1 - x) * Gf_H2O - 0.0591 * 2 * x / (1 - x) * pH - 2 * x / (1 - x) * V

    return fun

def opt_oxide_stability(args):
    # Brute force search for min since only 1D min problem
    x = np.linspace(0, 0.6, 7)
    f = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        f[i] = oxide_opt_obj_fun(x[i], *args)
    
    xmin = x[np.argmin(f)]
    fmin = np.min(f)

    # if xmin == x[-1]:
    #     print('Minimization likes super oxides - NN must not be accurate here.')
    #     xmin = x[0]
    #     fmin = f[0]

    return xmin, fmin


def total_gibbs(species_data, el_A, el_B, conc_A, conc_B, model, pH, V):
    Gf_AB_alloy = get_Gf_alloy(el_A, el_B, conc_A, conc_B, model, p_dict_oh_t)
    species_A = []
    species_B = []
    for data in species_data:
        if el_A in data['formula']:
            species_A.append(data)
        elif el_B in data['formula']:
            species_B.append(data)

    ge = []
    decomps = []
    for data_A in species_A:
        for data_B in species_B:
            if data_A['phase'] == 'ion' or data_B['phase'] == 'ion':
                phase = 'ion'
            else:
                phase = 'solid'
            decomps.append({'name': f"{data_A['name']} + {data_B['name']}", 'phase': phase})
            ge.append(get_dG(data_A, data_B, conc_A, conc_B, Gf_AB_alloy, pH, V))
    
    ge_array = np.array(ge)
    min_ge = np.min(ge_array)

    min_index = np.argmin(ge_array)
    min_decomp = decomps[min_index]

    return min_ge, min_decomp


# Convert the dictionary of NumPy arrays/lists into a dictionary of Torch tensors
p_dict_oh_t = {
    key: torch.tensor(value, dtype=torch.float32, device=device, requires_grad=True)
    for key, value in p_dict_oh.items()
}

el_A = 'Ti'
el_B = 'Mn'
conc_A = 0.2
conc_B = 1 - conc_A

pH = 14
V = 0

O_conc, ge_MO = opt_oxide_stability(args=(el_A, el_B, conc_A, conc_B, combined_model, p_dict_oh_t, pH, V))
min_ge, min_decomp = total_gibbs(species_data, el_A, el_B, conc_A, conc_B, combined_model, pH, V)

print(f'Most stable oxide: {el_A}{conc_A * (1 - O_conc):.2f}{el_B}{(1 - conc_A) * (1 - O_conc):.2f}O{O_conc:.2f} with modified free energy {ge_MO} eV')
print(f"Most stable decomposition: {min_decomp} with modified free energy {min_ge} eV")


Most stable oxide: Ti0.08Mn0.32O0.60 with modified free energy -4.469990999412537 eV
Most stable decomposition: {'name': 'TiO$_2$ + MnHO$_2$$^{-}$', 'phase': 'ion'} with modified free energy -4.328720523223877 eV


In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from adjustText import adjust_text

# ── Global matplotlib style (LaTeX fonts, larger text) ──────────────
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica", "DejaVu Sans"],
    "font.weight": "bold",
    "axes.labelweight": "bold",
    "axes.labelsize": 32,
    "axes.titlesize": 38,
    "xtick.labelsize": 28,
    "ytick.labelsize": 28,
})


# ── Convert your p‑dict to torch tensors (assumes p_dict_oh + device exist)
p_dict_oh_t = {
    k: torch.tensor(v, dtype=torch.float32, device=device, requires_grad=True)
    for k, v in p_dict_oh.items()
}

# Composition grid
conc_As = np.linspace(0.1, 0.9, 9)

# Master pH/E grid
NGrid    = 100
pH_range = np.linspace(-2, 16, NGrid)
V_range  = np.linspace(-2,  4,  NGrid)
pH_grid, V_grid = np.meshgrid(pH_range, V_range)

# Loop over compositions
for k, conc_A in enumerate(conc_As):
    el_A, el_B = "Ti", "Mn"
    conc_B     = 1 - conc_A

    # ── Stability analysis over the grid ────────────────────────────
    stable_species = []
    for i, pH in enumerate(pH_range):
        row_species = []
        for j, V in enumerate(V_range):
            # (1) Mixed‑oxide candidate
            O_conc, ge_MO = opt_oxide_stability(
                args=(el_A, el_B, conc_A, conc_B,
                      combined_model, p_dict_oh_t, pH, V)
            )
            O_string = (f"{el_A}$_{{{conc_A*(1-O_conc):.2f}}}$"
                        f"{el_B}$_{{{conc_B*(1-O_conc):.2f}}}$"
                        f"O$_{{{O_conc:.2f}}}$")
            species_O = {"name": O_string, "phase": "solid"}

            # (2) Minimal‑G decomposition
            min_ge, min_decomp = total_gibbs(
                species_data, el_A, el_B, conc_A, conc_B,
                combined_model, pH, V
            )

            row_species.append(species_O if ge_MO <= min_ge else min_decomp)
        stable_species.append(row_species)

    # ── Map each distinct phase to an integer ───────────────────────
    unique_species_info = {}
    for i in range(NGrid):
        for j in range(NGrid):
            s = stable_species[i][j]
            if s["name"] not in unique_species_info:
                unique_species_info[s["name"]] = {"phase": s["phase"]}

    species_names   = sorted(unique_species_info.keys())
    species_to_int  = {n: idx for idx, n in enumerate(species_names)}

    stable_int = np.zeros((NGrid, NGrid), dtype=int)
    for i in range(NGrid):
        for j in range(NGrid):
            stable_int[i, j] = species_to_int[stable_species[i][j]["name"]]
    stable_int = stable_int.T  # pcolormesh expects (x,y) orientation

    # ── Colour map: light‑blue = solid, beige = other ───────────────
    color_list = ["#ADD8E6" if unique_species_info[n]["phase"] == "solid"
                  else "#F5DEB3" for n in species_names]
    cmap  = mcolors.ListedColormap(color_list)
    bnds  = np.arange(stable_int.min()-0.5, stable_int.max()+1.5)
    norm  = mcolors.BoundaryNorm(bnds, len(color_list))

    # ── Plot ────────────────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.pcolormesh(pH_grid, V_grid, stable_int,
                  cmap=cmap, norm=norm, shading="nearest")

    # Label phases at centre of mass
    texts = []
    for idx, name in enumerate(species_names):
        mask = stable_int == idx
        if mask.any():
            y_idx, x_idx = np.where(mask)
            mean_pH = pH_range[int(round(x_idx.mean()))]
            mean_E  = V_range[int(round(y_idx.mean()))]
            texts.append(ax.text(mean_pH, mean_E, name,
                                 ha="center", va="center",
                                 fontsize=24, weight="bold"))
    adjust_text(texts, ax=ax, expand_points=(1.2, 1.4))

    # Water stability lines
    E_O2 = 1.229 - 0.0591 * pH_range
    E_H2 =        -0.0591 * pH_range
    ax.plot(pH_range, E_O2, "--", lw=2.0, color="red",  label=r"$\mathrm{O_2/H_2O}$")
    ax.plot(pH_range, E_H2, "--", lw=2.0, color="blue", label=r"$\mathrm{H^+/H_2}$")

    # Axes, limits, legend
    ax.set_xlim(-2, 16)
    ax.set_ylim(-2,  4)
    ax.set_xlabel(r"pH")
    ax.set_ylabel(r"V vs.\ SHE")
    # ax.set_title(fr"Ti$_{{{conc_A:.1f}}}$Mn$_{{{conc_B:.1f}}}$")

    fig.tight_layout()
    fig.savefig(f"./pourbaix_diagrams/TiMn_{k}.png", dpi=300)
    plt.close(fig)


In [6]:
import cv2
import os
import glob

# Set the folder containing the images
image_folder = "./pourbaix_diagrams"
video_name = "TiMn_pourbaix_sweep.mp4"
frame_rate = 20  # Adjust as needed

# Get all image files in sorted order
images = sorted(glob.glob(f"{image_folder}/*.png"))

# Read the first image to get dimensions
frame = cv2.imread(images[0])
height, width, layers = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' for .avi
video = cv2.VideoWriter(video_name, fourcc, frame_rate, (width, height))

# Add images to video
for image in images:
    video.write(cv2.imread(image))

# Release video writer
video.release()
cv2.destroyAllWindows()
